# Data Preprocessing and QC

In [1]:
import polars as pl

## 1. Loading Pillar Project Data

In [2]:
# pillar_condensed_df = pl.read_csv("../1_inputs/raw_inputs/pillar_data/pillar_data_condensed_051325_wREVEL.csv", schema_overrides={"Chrom": pl.String, 'aa_pos': pl.String, 'auth_reported_score': pl.String})
# pillar_condensed_df

pillar_clinvar_df = pl.read_csv("../1_inputs/raw_inputs/pillar_data/pillar_data_clinvar38_051325_wREVEL.csv", 
                                schema_overrides={"Chrom": pl.String, 'aa_pos': pl.String, 'auth_reported_score': pl.String})
print("Original pillar dataset shape:", pillar_clinvar_df.shape)

## filter the SNP variants only
pillar_clinvar_single_aa_df = pillar_clinvar_df.filter(
    (pl.col("consequence").str.contains("sense")) & \
    (pl.col("ref_allele").str.len_chars()==1) & (pl.col("alt_allele").str.len_chars()==1) & \
    (pl.col("hg38_start").is_not_null()) & (pl.col("hg38_end").is_not_null())
).drop_nulls(subset="hgvs_p").with_columns(
    pl.col("hg38_start").cast(pl.Int32).cast(pl.String).alias("nuc_loc")
)
print("Filtered missense/SNP pillar dataset shape:", pillar_clinvar_single_aa_df.shape)
display(pillar_clinvar_single_aa_df) #.filter(pl.col("Ref_seq_transcript_ID").str.contains("NC_"))
# pillar_clinvar_df.filter(pl.col("consequence").str.contains("nonsense"))["consequence"].unique()

Original pillar dataset shape: (1063392, 59)
Filtered missense/SNP pillar dataset shape: (128386, 60)


ID,Dataset,Gene,HGNC_id,Chrom,STRAND,hg19_pos,hg38_start,hg38_end,ref_allele,alt_allele,auth_transcript_id,transcript_pos,transcript_ref,transcript_alt,aa_pos,aa_ref,aa_alt,hgvs_c,hgvs_p,consequence,auth_reported_score,auth_reported_rep_score,auth_reported_func_class,splice_measure,gnomad_MAF,clinvar_sig,clinvar_star,clinvar_date_last_reviewed,nucleotide_or_aa,MaveDB URN (score set),Ensembl_transript_ID,Ref_seq_transcript_ID,Model_system,Assay_type,Phenotype_measured,Phenotype_detail,IGVF_produced,simplified_consequence,Flag,REVEL,Interval 1 name,Interval 1 range,Interval 1 MaveDB class,Interval 2 name,Interval 2 range,Interval 2 MaveDB class,Interval 3 name,Interval 3 range,Interval 3 MaveDB class,Interval 4 name,Interval 4 range,Interval 4 MaveDB class,Interval 5 name,Interval 5 range,Interval 5 MaveDB class,Interval 6 name,Interval 6 range,Interval 6 MaveDB class,nuc_loc
str,str,str,f64,str,f64,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.476163e6,3.476163e6,"""A""","""G""",null,"""4""",null,null,"""2""","""T""","""A""","""c.4A>G""","""p.Thr2Ala""","""missense_variant""","""0.9031""",null,null,"""No""",0.000002,null,null,null,"""aa""","""urn:mavedb:00000657-a-1""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Fluorescence""","""protein stability""","""No""","""missense_variant""",null,0.204,"""low abundance""","""(-Inf, 0.2)""","""Abnormal""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""3476163"""
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.476164e6,3.476164e6,"""C""","""T""",null,"""5""",null,null,"""2""","""T""","""I""","""c.5C>T""","""p.Thr2Ile""","""missense_variant""","""0.1793""",null,null,"""No""",0.000001,null,null,null,"""aa""","""urn:mavedb:00000657-a-1""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Fluorescence""","""protein stability""","""No""","""missense_variant""",null,0.323,"""low abundance""","""(-Inf, 0.2)""","""Abnormal""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""3476164"""
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.476164e6,3.476164e6,"""C""","""A""",null,"""5""",null,null,"""2""","""T""","""N""","""c.5C>A""","""p.Thr2Asn""","""missense_variant""","""0.948""",null,null,"""No""",null,null,null,null,"""aa""","""urn:mavedb:00000657-a-1""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Fluorescence""","""protein stability""","""No""","""missense_variant""",null,0.256,"""low abundance""","""(-Inf, 0.2)""","""Abnormal""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""3476164"""
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.476163e6,3.476163e6,"""A""","""C""",null,"""4""",null,null,"""2""","""T""","""P""","""c.4A>C""","""p.Thr2Pro""","""missense_variant""","""1.1182""",null,null,"""No""",null,null,null,null,"""aa""","""urn:mavedb:00000657-a-1""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Fluorescence""","""protein stability""","""No""","""missense_variant""",null,0.537,"""low abundance""","""(-Inf, 0.2)""","""Abnormal""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""3476163"""
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.476163e6,3.476163e6,"""A""","""T""",null,"""4""",null,null,"""2""","""T""","""S""","""c.4A>T""","""p.Thr2Ser""","""missense_variant""","""1.0407""",null,

In [3]:
pillar_summary_df = pl.read_csv("../1_inputs/raw_inputs/pillar_data/summary_pp_df_051325_expanded.tsv", separator="\t", schema_overrides={"Chrom": pl.String, 'aa_pos': pl.String, 'auth_reported_score': pl.String})
pillar_summary_df.head()

Dataset,Total_Variants,gnomAD,Pathogenic,Benign,Synonymous,Stop
str,i64,i64,i64,i64,i64,i64
"""ASPA_Grønbæk-Thygesen_2024_abu…",6151,365,67,6,0,915
"""ASPA_Grønbæk-Thygesen_2024_tox…",6148,365,67,6,0,915
"""BAP1_Waters_2024""",18108,1871,202,1060,1854,922
"""BARD1_unpublished""",8957,2006,129,980,1233,224
"""BRCA1_Adamovich_2022_Cisplatin""",1427,87,43,47,0,123


## 2. Loading VarChAMP data

In [4]:
def map_chr(num):
    if num == 23:
        return "X"
    elif num == 24:
        return "Y"
    elif num == 12920:
        return "MT"
    else:
        return str(num)
        

varchamp_seq_df = pl.read_csv("../1_inputs/VarChampSeqConfirmationResult.tsv", separator="\t")
varchamp_seq_df = varchamp_seq_df.with_columns(
    pl.concat_str([pl.col("symbol"),pl.col("aa_change")], separator="_").alias("gene_variant")
).with_columns(
    pl.col("spdi")
    .str.extract(r"NC_0*([0-9]+)", 1)
    .cast(pl.Int32)
    .alias("chr_num")
).with_columns(
    pl.col("chr_num")
    .map_elements(map_chr, return_dtype=pl.String)
    .alias("Chrom")
).with_columns(
    pl.col("spdi").str.split(":").list.get(1).alias("nuc_loc"),
    pl.col("spdi").str.split(":").list.get(2).alias("ref_allele"),
    pl.col("spdi").str.split(":").list.get(3).alias("alt_allele"),
)
varchamp_seq_df

# varchamp_seq_df["mutation_id_old"].unique()
# varchamp_seq_df.filter((pl.col("nuc_ref").str.len_chars()>1)|(pl.col("nuc_alt").str.len_chars()>1))
# varchamp_seq_df.unique(subset="gene_variant")

symbol,ensembl_gene_id,orf_id_wt,mutation_id_old,ccsb_mutation_id,ccsb_allele_id,spdi,nt_change,aa_change,collection,entry_plate_orig,entry_well_orig,entry_plate_conso,entry_well_conso,entry_seq_pool,db_plate,db_well,n2h_plate,n2h_well,dualip_plate,dualip_well,mislocalization_plate,mislocalization_well,entry_sequenced,entry_sequence_confirmation_class,db_sequenced,db_sequence_confirmation_class,n2h_sequenced,n2h_sequence_confirmation_class,dualip_sequenced,dualip_sequence_confirmation_class,mislocalization_sequenced,mislocalization_sequence_confirmation_class,gene_variant,chr_num,Chrom,nuc_loc,ref_allele,alt_allele
str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,i32,str,str,str,str
"""GBA1""","""ENSG00000177628""",2,6,"""CCSBVarC000001""","""ALE0000584""","""NC_000001.11:155240033:C:G""","""160G>C""","""Val54Leu""","""RC4""","""RC4_Mut_GDEh1026""","""H01""","""GDEhDisVCh_40054""","""F12""","""2""","""RC4_Mut_GDDh1026""","""H01""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""1""",1,"""2""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""GBA1_Val54Leu""",1,"""1""","""155240033""","""C""","""G"""
"""GBA1""","""ENSG00000177628""",2,73,"""CCSBVarC000002""","""ALE00000002""","""NC_000001.11:155238225:G:A""","""670C>T""","""Leu224Phe""","""RC4""","""RC4_Mut_GDEh1026""","""E01""","""GDEhDisVCh_40054""","""C12""","""2""","""RC4_Mut_GDDh1026""","""E01""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""1""",1,"""1""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""GBA1_Leu224Phe""",1,"""1""","""155238225""","""G""","""A"""
"""GBA1""","""ENSG00000177628""",2,113,"""CCSBVarC000003""","""ALE00000003""","""NC_000001.11:155237453:C:T""","""887G>A""","""Arg296Gln""","""RC4""","""RC4_Mut_GDEh1026""","""F01""","""GDEhDisVCh_40054""","""D12""","""2""","""RC4_Mut_GDDh1026""","""F01""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""1""",1,"""7""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""GBA1_Arg296Gln""",1,"""1""","""155237453""","""C""","""T"""
"""GBA1""","""ENSG00000177628""",2,231,"""CCSBVarC000004""","""ALE00000004""","""NC_000001.11:155235252:A:G""","""1448T>C""","""Leu483Pro""","""RC4""","""RC4_Mut_GDEh1026""","""G01""","""GDEhDisVCh_40054""","""E12""","""2""","""RC4_Mut_GDDh1026""","""G01""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""1""",1,"""2""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""GBA1_Leu483Pro""",1,"""1""","""155235252""","""A""","""G"""
"""GBA1""","""ENSG00000177628""",2,213510,"""CCSBVarC003869""","""ALE00003869""","""NC_000001.11:155239934:G:A""","""259C>T""","""Arg87Trp""","""CEGS2""","""CegsMutGDEh1035""","""B03""","""GDEhDisVCh_40054""","""B02""","""2""","""CegsMutGDDh1035""","""B03""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""1""",1,"""2""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""GBA1_Arg87Trp""",1,"""1""","""155239934""","""G""","""A"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""CTNNB1""","""ENSG00000168036""",100070227,212487,"""CCSBVarC007073""","""ALE00007071""","""NC_000003.12:41225785:A:G""","""860A>G""","""Asn287Ser""","""CEGS2""","""CegsMutGDEh1023""","""B07""","""GDEhDisVCh_40034""","""E09""","""2""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""1""",0,"""NULL""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""CTNNB1_Asn287Ser""",3,"""3""","""41225785""","""A""","""G"""
"""TBX19""","""ENSG00000143178""",100070258,212488,"""CCSBVarC007074""","""ALE00007072""","""NC_000001.11:168291339:C:T""","""383C>T""","""Ser128Phe""","""CEGS2""","""CegsMutGDEh1043""","""A07""","""GDEhDisVCh_40046""","""B02""","""2""","""CegsMutGDDh1043""","""A07""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""7""",1,"""7""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""TBX19_Ser128Phe""",1,"""1""","""168291339""","""C""","""T"""
"""TBX19""","

In [5]:
# varchamp_consolid_seq_df = pl.read_csv("../1_inputs/VarChamp_Consolidated_allele_collection.tsv", separator="\t", infer_schema_length=10000)
# varchamp_consolid_seq_df
# varchamp_consolid_seq_df = pl.read_csv("../1_inputs/varchamp_consolidated_alleles_clinvar_gnomad_dbnsfp_scores.tsv", separator="\t", infer_schema_length=10000)
# varchamp_consolid_seq_df_sc = varchamp_consolid_seq_df.join(
#     varchamp_seq_df,
#     on="spdi",
#     how="left"
# )
# varchamp_consolid_seq_df_sc

## 3. Attempt to merge two datasets

In [6]:
## QC
# aa_single_to_three = {
#     'A': 'Ala',  # Alanine
#     'R': 'Arg',  # Arginine
#     'N': 'Asn',  # Asparagine
#     'D': 'Asp',  # Aspartic acid
#     'C': 'Cys',  # Cysteine
#     'Q': 'Gln',  # Glutamine
#     'E': 'Glu',  # Glutamic acid
#     'G': 'Gly',  # Glycine
#     'H': 'His',  # Histidine
#     'I': 'Ile',  # Isoleucine
#     'L': 'Leu',  # Leucine
#     'K': 'Lys',  # Lysine
#     'M': 'Met',  # Methionine
#     'F': 'Phe',  # Phenylalanine
#     'P': 'Pro',  # Proline
#     'S': 'Ser',  # Serine
#     'T': 'Thr',  # Threonine
#     'W': 'Trp',  # Tryptophan
#     'Y': 'Tyr',  # Tyrosine
#     'V': 'Val',   # Valine
#     '*': '*'
# }

pillar_clinvar_single_aa_df = pillar_clinvar_single_aa_df.with_columns(
    pl.col("hgvs_p").str.split(".").list.get(-1).alias("aa_change")
).with_columns(
    pl.concat_str([pl.col("Gene"),pl.col("aa_change")], separator="_").alias("gene_variant")
).sort("gene_variant", "gnomad_MAF", descending=[False,False])

print(pillar_clinvar_single_aa_df.unique(subset="gene_variant"))
pillar_clinvar_single_aa_df

shape: (61_155, 62)
┌────────────┬───────────┬───────────┬─────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ ID         ┆ Dataset   ┆ Gene      ┆ HGNC_id ┆ … ┆ Interval  ┆ nuc_loc   ┆ aa_change ┆ gene_vari │
│ ---        ┆ ---       ┆ ---       ┆ ---     ┆   ┆ 6 MaveDB  ┆ ---       ┆ ---       ┆ ant       │
│ str        ┆ str       ┆ str       ┆ f64     ┆   ┆ class     ┆ str       ┆ str       ┆ ---       │
│            ┆           ┆           ┆         ┆   ┆ ---       ┆           ┆           ┆ str       │
│            ┆           ┆           ┆         ┆   ┆ str       ┆           ┆           ┆           │
╞════════════╪═══════════╪═══════════╪═════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ PTEN_Mighe ┆ PTEN_Migh ┆ PTEN      ┆ 9588.0  ┆ … ┆ null      ┆ 87925555  ┆ Asn69Lys  ┆ PTEN_Asn6 │
│ ll_2018_va ┆ ell_2018  ┆           ┆         ┆   ┆           ┆           ┆           ┆ 9Lys      │
│ r89064     ┆           ┆           ┆         ┆   ┆           ┆       

ID,Dataset,Gene,HGNC_id,Chrom,STRAND,hg19_pos,hg38_start,hg38_end,ref_allele,alt_allele,auth_transcript_id,transcript_pos,transcript_ref,transcript_alt,aa_pos,aa_ref,aa_alt,hgvs_c,hgvs_p,consequence,auth_reported_score,auth_reported_rep_score,auth_reported_func_class,splice_measure,gnomad_MAF,clinvar_sig,clinvar_star,clinvar_date_last_reviewed,nucleotide_or_aa,MaveDB URN (score set),Ensembl_transript_ID,Ref_seq_transcript_ID,Model_system,Assay_type,Phenotype_measured,Phenotype_detail,IGVF_produced,simplified_consequence,Flag,REVEL,Interval 1 name,Interval 1 range,Interval 1 MaveDB class,Interval 2 name,Interval 2 range,Interval 2 MaveDB class,Interval 3 name,Interval 3 range,Interval 3 MaveDB class,Interval 4 name,Interval 4 range,Interval 4 MaveDB class,Interval 5 name,Interval 5 range,Interval 5 MaveDB class,Interval 6 name,Interval 6 range,Interval 6 MaveDB class,nuc_loc,aa_change,gene_variant
str,str,str,f64,str,f64,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ASPA_Grønbæk-Thygesen_2024_tox…","""ASPA_Grønbæk-Thygesen_2024_tox…","""ASPA""",756.0,"""17""",1.0,null,3.483509e6,3.483509e6,"""C""","""A""",null,"""443""",null,null,"""148""","""A""","""D""","""c.443C>A""","""p.Ala148Asp""","""missense_variant""","""-0.1191""",null,null,"""No""",6.1958e-7,null,null,null,"""aa""","""urn:mavedb:00000657-a-2""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Cell viability""","""protein toxicity""","""No""","""missense_variant""",null,0.714,"""Toxic""","""(0.6, Inf)""","""Abnormal""","""Non-toxic""","""(-Inf, 0.4)""","""Normal""",null,null,null,null,null,null,null,null,null,null,null,null,"""3483509""","""Ala148Asp""","""ASPA_Ala148Asp"""
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.483509e6,3.483509e6,"""C""","""A""",null,"""443""",null,null,"""148""","""A""","""D""","""c.443C>A""","""p.Ala148Asp""","""missense_variant""","""0.9495""",null,null,"""No""",6.1958e-7,null,null,null,"""aa""","""urn:mavedb:00000657-a-1""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Fluorescence""","""protein stability""","""No""","""missense_variant""",null,0.714,"""low abundance""","""(-Inf, 0.2)""","""Abnormal""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""3483509""","""Ala148Asp""","""ASPA_Ala148Asp"""
"""ASPA_Grønbæk-Thygesen_2024_tox…","""ASPA_Grønbæk-Thygesen_2024_tox…","""ASPA""",756.0,"""17""",1.0,null,3.483509e6,3.483509e6,"""C""","""G""",null,"""443""",null,null,"""148""","""A""","""G""","""c.443C>G""","""p.Ala148Gly""","""missense_variant""","""0.1248""",null,null,"""No""",null,null,null,null,"""aa""","""urn:mavedb:00000657-a-2""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Cell viability""","""protein toxicity""","""No""","""missense_variant""",null,0.679,"""Toxic""","""(0.6, Inf)""","""Abnormal""","""Non-toxic""","""(-Inf, 0.4)""","""Normal""",null,null,null,null,null,null,null,null,null,null,null,null,"""3483509""","""Ala148Gly""","""ASPA_Ala148Gly"""
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.483509e6,3.483509e6,"""C""","""G""",null,"""443""",null,null,"""148""","""A""","""G""","""c.443C>G""","""p.Ala148Gly""","""missense_variant""","""0.2662""",null,null,"""No""",null,null,null,null,"""aa""","""urn:mavedb:00000657-a-1""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Fluorescence""","""protein stability""","""No""","""missense_variant""",null,0.679,"""low abundance""","""(-Inf, 0.2)""","""Abnormal""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""3483509""","""Ala148Gly""","""ASPA_Ala148Gly"""

### 3.1 Merge by gene_variant

In [10]:
pillar_varchamp_df = pillar_clinvar_single_aa_df.join(
    varchamp_seq_df,
    on="gene_variant" #["Chrom", "nuc_loc", "ref_allele", "alt_allele"]
)
pillar_varchamp_df#.unique(subset="gene_variant")

pillar_varchamp_df.write_csv("../3_outputs/pillar_varchamp_overlapped_variants.tsv", separator="\t") #.unique(subset="spdi")

### 3.2 Merge by genomic locations

In [8]:
pillar_varchamp_df = pillar_clinvar_single_aa_df.join(
    varchamp_seq_df,
    on=["Chrom", "nuc_loc", "ref_allele", "alt_allele"]
)
pillar_varchamp_df

ID,Dataset,Gene,HGNC_id,Chrom,STRAND,hg19_pos,hg38_start,hg38_end,ref_allele,alt_allele,auth_transcript_id,transcript_pos,transcript_ref,transcript_alt,aa_pos,aa_ref,aa_alt,hgvs_c,hgvs_p,consequence,auth_reported_score,auth_reported_rep_score,auth_reported_func_class,splice_measure,gnomad_MAF,clinvar_sig,clinvar_star,clinvar_date_last_reviewed,nucleotide_or_aa,MaveDB URN (score set),Ensembl_transript_ID,Ref_seq_transcript_ID,Model_system,Assay_type,Phenotype_measured,Phenotype_detail,…,aa_change,gene_variant,symbol,ensembl_gene_id,orf_id_wt,mutation_id_old,ccsb_mutation_id,ccsb_allele_id,spdi,nt_change,aa_change_right,collection,entry_plate_orig,entry_well_orig,entry_plate_conso,entry_well_conso,entry_seq_pool,db_plate,db_well,n2h_plate,n2h_well,dualip_plate,dualip_well,mislocalization_plate,mislocalization_well,entry_sequenced,entry_sequence_confirmation_class,db_sequenced,db_sequence_confirmation_class,n2h_sequenced,n2h_sequence_confirmation_class,dualip_sequenced,dualip_sequence_confirmation_class,mislocalization_sequenced,mislocalization_sequence_confirmation_class,gene_variant_right,chr_num
str,str,str,f64,str,f64,str,f64,f64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,f64,str,str,str,str,str,str,str,str,str,str,str,…,str,str,str,str,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,i32
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.47623e6,3.47623e6,"""A""","""G""",null,"""71""",null,null,"""24""","""E""","""G""","""c.71A>G""","""p.Glu24Gly""","""missense_variant""","""-0.0016""",null,null,"""No""",6.1951e-7,"""Likely pathogenic""","""criteria provided, single subm…","""Dec 13, 2023""","""aa""","""urn:mavedb:00000657-a-1""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Fluorescence""","""protein stability""",…,"""Glu24Gly""","""ASPA_Glu24Gly""","""ASPA""","""ENSG00000108381""",8459,15684,"""CCSBVarC001383""","""ALE00001383""","""NC_000017.11:3476230:A:G""","""71A>G""","""Glu24Gly""","""RC4""","""RC4_Mut_GDEh1016""","""D01""","""GDEhDisVCh_40040""","""C04""","""2""","""RC4_Mut_GDDh1016""","""D01""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""7""",1,"""1""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""ASPA_Glu24Gly""",17
"""ASPA_Grønbæk-Thygesen_2024_tox…","""ASPA_Grønbæk-Thygesen_2024_tox…","""ASPA""",756.0,"""17""",1.0,null,3.47623e6,3.47623e6,"""A""","""G""",null,"""71""",null,null,"""24""","""E""","""G""","""c.71A>G""","""p.Glu24Gly""","""missense_variant""","""0.2914""",null,null,"""No""",6.1951e-7,"""Likely pathogenic""","""criteria provided, single subm…","""Dec 13, 2023""","""aa""","""urn:mavedb:00000657-a-2""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Cell viability""","""protein toxicity""",…,"""Glu24Gly""","""ASPA_Glu24Gly""","""ASPA""","""ENSG00000108381""",8459,15684,"""CCSBVarC001383""","""ALE00001383""","""NC_000017.11:3476230:A:G""","""71A>G""","""Glu24Gly""","""RC4""","""RC4_Mut_GDEh1016""","""D01""","""GDEhDisVCh_40040""","""C04""","""2""","""RC4_Mut_GDDh1016""","""D01""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""","""NULL""",1,"""7""",1,"""1""",0,"""NULL""",0,"""NULL""",0,"""NULL""","""ASPA_Glu24Gly""",17
"""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA_Grønbæk-Thygesen_2024_abu…","""ASPA""",756.0,"""17""",1.0,null,3.499e6,3.499e6,"""A""","""C""",null,"""854""",null,null,"""285""","""E""","""A""","""c.854A>C""","""p.Glu285Ala""","""missense_variant""","""0.9765""",null,null,"""No""",0.000221,"""Pathogenic""","""criteria provided, multiple su…","""Mar 30, 2024""","""aa""","""urn:mavedb:00000657-a-1""","""ENST00000263080.3""","""NM_000049.4""","""immortalized human cells""","""Reporter""","""Fluorescence""","""protein stability""",…,"""Glu285Ala""","""ASPA_Glu285Ala""","""ASPA""","""ENSG00000108381""",8459,15721,"""CCSBVarC0013